Coverting the .CSV file into a .TSV

In [ ]:
import csv
import pandas as pd # type: ignore
import requests # type: ignore
from tqdm import tqdm # type: ignore


In [3]:
# Convert TSV file to CSV

def convert_tsv_to_csv(tsv_file, csv_file):
    with open(tsv_file, 'r', newline='', encoding='utf-8') as tsv:
        reader = csv.reader(tsv, delimiter='\t')
        with open(csv_file, 'w', newline='', encoding='utf-8') as csv_out:
            writer = csv.writer(csv_out, delimiter=',')
            for row in reader:
                writer.writerow(row)

convert_tsv_to_csv('v2.tsv','homework_2.csv')

In [5]:
# Load the CSV file
file_path = "homework_2.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Function to generate image captions based on the conditions
def generate_caption(row):
    caption = f"A mysterious event in {row['location']}, {row['city']}, {row['state']}. "
    
    if pd.notna(row.get("Event")):
        caption += f"The event was classified as {row['Event'].lower()}. "
    
    if pd.notna(row.get("Apparition Type")) and row["Apparition Type"].lower() != "unknown":
        caption += f"An apparition of type {row['Apparition Type'].lower()} was reported. "
    
    if pd.notna(row.get("Apparition Gender")) and row["Apparition Gender"].lower() != "unknown":
        caption += f"The apparition appeared to be {row['Apparition Gender'].lower()}. "

    if pd.notna(row.get("time_of_day")):
        caption += f"The sighting occurred during {row['time_of_day'].lower()}. "

    if pd.notna(row.get("Witness Count")):
        caption += f"{row['Witness Count']} witness(es) reported the phenomenon. "

    if row.get("Audio Evidence") == "Yes":
        caption += f"Audio evidence was captured. "
        if pd.notna(row.get("Audio Reasoning")):
            caption += f"The sound analysis suggested: {row['Audio Reasoning']}. "

    if row.get("Visual Evidence") == True:
        caption += f"Visual evidence was obtained. "
        if pd.notna(row.get("Visual Reasoning")):
            caption += f"Experts noted: {row['Visual Reasoning']}. "

    return caption.strip()

# Applying the function to generate captions
df["Image Caption"] = df.apply(generate_caption, axis=1)

# Save the results to a new CSV file
output_file = "image_captions.csv"
df.to_csv(output_file, index=False)

print(f"Image captions saved to {output_file}")


Image captions saved to image_captions.csv


In [ ]:
import base64
import requests # type: ignore
import os

url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"

body = {
  "steps": 40,
  "width": 1024,
  "height": 1024,
  "seed": 0,
  "cfg_scale": 5,
  "samples": 1,
  "text_prompts": [
    {
      "text": "A painting of a cat",
      "weight": 1
    },
    {
      "text": "blurry, bad",
      "weight": -1
    }
  ],
}

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "sk-m8GS6Yg8PE7ab93SLBmd7kKRwgMHlD9zqf5J3o3s9PH8ZbbX",
}

response = requests.post(
  url,
  headers=headers,
  json=body,
)

if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

# make sure the out directory exists
if not os.path.exists("./out"):
    os.makedirs("./out")

for i, image in enumerate(data["artifacts"]):
    with open(f'./out/txt2img_{image["seed"]}.png', "wb") as f:
        f.write(base64.b64decode(image["base64"]))

In [2]:
import pandas as pd
import requests
from tqdm import tqdm

def get_best_caption(image_path, beam_size=6):
    # Use endpoint to change the beamsize/output
    TIKA_CAPTION_ENDPOINT = f"http://localhost:8764/inception/v3/caption/image?beam_size={beam_size}"
    try:
        with open(image_path, 'rb') as f:
            response = requests.post(
                TIKA_CAPTION_ENDPOINT,
                headers={'Accept': 'application/json'},
                data=f
            )
    except Exception as e:
        return f"Exception when processing {image_path}: {e}"

    if response.status_code == 200:
        try:
            data = response.json()
            captions = data.get('captions', [])
            if captions:
                # pick up only most confident/relevant caption
                best_caption = max(captions, key=lambda x: x["confidence"])
                return best_caption['sentence']
            else:
                return "No valid caption found"
        except ValueError:
            return "Failed to parse JSON"
    else:
        return f"Error {response.status_code} - {response.text}"

# load in the dataset
# CHANGE TO YOUR RESPECTIVE CSV PATH
file_path = "image_captions_with_paths_desc.csv"
df = pd.read_csv(file_path)

# Uncomment the following line to use a random sample of 20 rows
# df = df.sample(20, random_state=25)

# process each image in the dataset with a progress bar
captions = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
    # PUT IN REAL IMG PATH/COL NAME FROM DF
    image_path = row['Image Path']
    caption = get_best_caption(image_path)
    captions.append(caption)

# add captions to the DF. Name col as please.
df['caption'] = captions

# Save the updated DF to CSV
output_path = "hw_2_with_images_and_captions.csv"
df.to_csv(output_path, index=False)

Processing images: 100%|██████████| 10980/10980 [3:09:39<00:00,  1.04s/it] 


In [6]:
import os
import requests
import pandas as pd
from tqdm import tqdm

# --- Configuration ---
CSV_PATH = 'hw_2_with_images_and_captions.csv'  # your input CSV file
OUTPUT_PATH = 'hw2_with_captions_and_objects.csv'  # output CSV
OBJECTS_ENDPOINT = 'http://localhost:8764/inception/v4/classify/image'

# --- Object Detection Function ---
def get_objects(image_path):
    try:
        with open(image_path, 'rb') as f:
            response = requests.post(
                OBJECTS_ENDPOINT,
                headers={'Accept': 'application/json'},
                data=f
            )
        if response.status_code == 200:
            data = response.json()
            return ', '.join(data.get('classnames', []))
        else:
            print(f"Object detection error {response.status_code} for {image_path}")
            return ''
    except Exception as e:
        print(f"Object detection exception for {image_path}: {e}")
        return ''

# --- Step 1: Read CSV ---
df = pd.read_csv(CSV_PATH, encoding='utf-8')

# --- Step 2: Process Objects ---
objects = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
    # Assumes you have a column named 'Image Path'
    image_path = row['Image Path']
    
    if os.path.exists(image_path):
        detected_objects = get_objects(image_path)
    else:
        print(f"Image not found: {image_path}")
        detected_objects = ''
    objects.append(detected_objects)

# --- Step 3: Save ---
df['objects'] = objects
df.to_csv(OUTPUT_PATH, index=False)
print(f"Objects saved to {OUTPUT_PATH}")


Processing images: 100%|██████████| 10980/10980 [54:28<00:00,  3.36it/s] 


Objects saved to hw2_with_captions_and_objects.csv
